# Train Network

This notebook trains a Convolutional Neural Network using categorized images.

### Install Packages

This notebook requires `Pillow`, `tensorflow` and `keras`. You may uncomment and run the cell below to have Jupyter Notebook install these for you.

In [1]:
# !pip install Pillow, tensorflow, keras

### Configuration

Set the variables below to specify where the categorical images are located, which classes to train and what the filename should be for saved training data. If you would like to train all 45+ categories, set `classes = None`.

In [8]:
import os

image_path = 'capture_data'

# Uncomment this to train all categories
classes = os.listdir(image_path)
checkpoint_path = 'weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5'

In [14]:
import matplotlib.image as mpimg
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
import Image
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(64, 64, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(classes), activation='softmax'))
    return model

In [11]:
def train():
    from keras.preprocessing.image import ImageDataGenerator
    from keras.callbacks import ModelCheckpoint

    model = create_model()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max',period=1)

    datagen = ImageDataGenerator(rescale = 1.0 / 255.0,  
                                 rotation_range=10, width_shift_range=0.2, height_shift_range=0.2,
                                 zoom_range=0.2,shear_range=20,
                                 validation_split=0.2)

    training_set = datagen.flow_from_directory(image_path, classes=classes, target_size=(64, 64), 
                                               color_mode='grayscale', batch_size=32, shuffle=True,
                                               subset="training")
    validation_set = datagen.flow_from_directory(image_path, classes=classes, target_size=(64, 64), 
                                               color_mode='grayscale', batch_size=32, shuffle=True,
                                               subset="validation")

    history = model.fit_generator(training_set,
                                  validation_data=validation_set,
                                  epochs=10, verbose=1, callbacks=[ checkpoint ])

Found 174 images belonging to 7 classes.
Found 40 images belonging to 7 classes.
Epoch 1/10
6/6 [==============================] - 5s 822ms/step - loss: 1.8799 - acc: 0.3511 - val_loss: 1.7906 - val_acc: 0.4250

Epoch 00001: val_acc improved from -inf to 0.42500, saving model to weights-improvement-01-0.42.hdf5
Epoch 2/10
6/6 [==============================] - 4s 659ms/step - loss: 1.8147 - acc: 0.3876 - val_loss: 1.7233 - val_acc: 0.4250

Epoch 00002: val_acc did not improve
Epoch 3/10
6/6 [==============================] - 4s 637ms/step - loss: 1.6948 - acc: 0.4044 - val_loss: 1.5886 - val_acc: 0.4250

Epoch 00003: val_acc did not improve
Epoch 4/10
6/6 [==============================] - 4s 634ms/step - loss: 1.6718 - acc: 0.4097 - val_loss: 1.4971 - val_acc: 0.5250

Epoch 00004: val_acc improved from 0.42500 to 0.52500, saving model to weights-improvement-04-0.53.hdf5
Epoch 5/10
6/6 [==============================] - 4s 670ms/step - loss: 1.5741 - acc: 0.4524 - val_loss: 1.3988 - va

In [17]:
def load_trained_model(weightfile):
    from keras.models import load_model
    model = load_model(weightfile)
    return model

In [18]:
trained_model = load_trained_model("weights-improvement-10-0.55.hdf5")


In [33]:
import cv2
import numpy as np
image = cv2.imread("capture_data/forward/Groot1529091688.png", 0)
image = np.array([ image.reshape(64, 64, 1) ])
print(classes[trained_model.predict_classes(image)[0]])

left
